**Assignment 2**

*CNN for Facial Expression classification*

In [ ]:
# Importing needed libraries and modules
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adadelta, Adam
from keras.constraints import maxnorm
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers.experimental.preprocessing import RandomFlip, RandomRotation, RandomContrast,RandomZoom,Resizing,RandomTranslation
from keras.utils import np_utils
from google.colab import drive

In [ ]:
# mount the drive
drive.mount('/gdrive')
%cd /gdrive

# Go to the assignment folder in the drive and make it current location
%cd /gdrive/My\ Drive/Assignment_2_data
print('Files in the Assignment 2 Data folder: ')
!ls

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive
/gdrive/My Drive/Assignment_2_data
Files in the Assignment 2 Data folder: 
best-model.h5  submission.csv  test_data.csv  train_data.csv  train_target.csv


In [ ]:
# loading the data 
X_train = np.array(pd.read_csv('train_data.csv', header=None))
X_test = np.array(pd.read_csv('test_data.csv',header=None))
Y_train = np.array(pd.read_csv('train_target.csv',header=None))

# Reshape the np array
X_train = X_train.reshape(-1,48,48)
X_test = X_test.reshape(-1,48,48)

# #normalize the inputs from 0-255 to 0-1
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

# Making sure everything is same dims
X_train = np.expand_dims(X_train,-1)
X_test = np.expand_dims(X_test,-1)

# print(X_train.shape,X_test.shape,Y_train.shape)

In [ ]:
# from sklearn.model_selection import train_test_split
# X_train,x_test,Y_train,Y_test = train_test_split(X_train,Y_train,test_size=0.2, random_state = 42)

In [ ]:
# One hot coding 
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
Y_train = lb.fit_transform(Y_train)
class_num = Y_train.shape[1]
# print(Y_train)

In [ ]:
# Creating the model
model = Sequential()

# Adding data augmentation
model.add(RandomContrast([0.5,1.5]))
model.add(RandomFlip('horizontal_and_vertical'))
model.add(RandomRotation([0.5,1.5]))
model.add(Resizing(128,128,interpolation='bicubic'))
model.add(RandomZoom([0.2,0.5]))
model.add(RandomTranslation(height_factor=(-0.2,0.2), width_factor=(-0.2,0.2)))

#first convolutional layer
model.add(Conv2D(32, (5,5), input_shape=(48,48,1), activation='relu', padding='same'))
# model.add(Conv2D(32, (5,5), input_shape=(48,48,1), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())

#second convolutional layer
model.add(Conv2D(64,(5,5), activation='relu', padding='same'))
model.add(Conv2D(64,(5,5), activation='relu', padding='same'))
# model.add(Conv2D(64,(5,5), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())

#third later 
model.add(Conv2D(128, (5,5), activation='relu', padding='same'))
# model.add(Conv2D(128, (5,5), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())

# fourth later 
model.add(Conv2D(256, (5,5), activation='relu', padding='same'))
model.add(Conv2D(256, (5,5), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())

# Fifth Layer 
model.add(Conv2D(512, (5,5), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dropout(0.2))

model.add(Dense(128, kernel_regularizer=l2(0.001),kernel_constraint=maxnorm(class_num)))
model.add(Activation('relu'))
    
model.add(Dense(32, kernel_regularizer=l2(0.001), kernel_constraint=maxnorm(class_num)))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(class_num))
model.add(Activation('softmax'))

In [ ]:
# np.random.seed(95)

# saving the model for later. Used in model checkpoint
bestModel = "best-model.h5"

model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
mc = ModelCheckpoint(bestModel, monitor='val_accuracy', save_freq='epoch', mode='max', save_best_only=True)
hist = model.fit(X_train, Y_train, validation_split= 0.1, epochs=300,batch_size=82, verbose=1, callbacks=[mc])

# Print out the highest accuracy we hit for the best model checkpoint
val = hist.history['val_accuracy']
print("Max Accuracy: ", np.max(val))


Epoch 1/300
178/178 [==============================] - 25s 139ms/step - loss: 1.3912 - accuracy: 0.4318 - val_loss: 1.2949 - val_accuracy: 0.4481
Epoch 2/300
178/178 [==============================] - 24s 134ms/step - loss: 1.2428 - accuracy: 0.4443 - val_loss: 1.1920 - val_accuracy: 0.4481
Epoch 3/300
178/178 [==============================] - 24s 137ms/step - loss: 1.1610 - accuracy: 0.4456 - val_loss: 1.1323 - val_accuracy: 0.4481
Epoch 4/300
178/178 [==============================] - 24s 134ms/step - loss: 1.1201 - accuracy: 0.4458 - val_loss: 1.1014 - val_accuracy: 0.4481
Epoch 5/300
178/178 [==============================] - 24s 133ms/step - loss: 1.0982 - accuracy: 0.4454 - val_loss: 1.0921 - val_accuracy: 0.4481
Epoch 6/300
178/178 [==============================] - 24s 134ms/step - loss: 1.0858 - accuracy: 0.4457 - val_loss: 1.0819 - val_accuracy: 0.4481
Epoch 7/300
178/178 [==============================] - 24s 134ms/step - loss: 1.0787 - accuracy: 0.4458 - val_loss: 1.0740 -

In [ ]:
# Loading the best model we saved
from keras.models import load_model
bestmodel = load_model('best-model.h5')

# Predicting using the saved model. 
scores = bestmodel.predict(X_test)
# print(scores)

# Getting the index of highest value and storing it into .csv file
scores = np.argmax(scores, axis=1)
scores = pd.Series(scores, name='Category',dtype=np.int32)
submission = pd.concat([scores], axis=1)
submission.to_csv("submission.csv", index_label='id')

# print scores to check with submission.csv if first 5 and last 5 match. 
print(scores)

0       0
1       0
2       1
3       1
4       0
       ..
3960    0
3961    0
3962    1
3963    2
3964    2
Name: Category, Length: 3965, dtype: int32
